In [40]:
import numpy as np
import pandas as pd
import hyperopt

from hyperopt import hp, fmin, tpe

from time import time

from sklearn.metrics import accuracy_score, roc_auc_score

%run ../src/Munger.py
%run ../src/Models.py
%run ../src/utils.py

In [34]:
# load dataset
numer_ai = pd.read_csv('../data/numerai_datasets/numerai_training_data.csv')
test = pd.read_csv('../data/numerai_datasets/numerai_tournament_data.csv')

In [35]:
# split into training and validation sets as per the competition instructions
train = numer_ai[numer_ai.validation == 0]
validation = numer_ai[numer_ai.validation == 1]

In [38]:
munger = Munger(train, validation, test)
munger.remove_correlated_features()
munger.label_encoding()

C:\Users\Abhishek\Anaconda2\lib\site-packages\pandas\core\indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Abhishek\Anaconda2\lib\site-packages\pandas\core\indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [39]:
X_train = munger.X
y_train = munger.y

X_validation = munger.X_validation
y_validation = munger.y_validation

X_test = munger.X_test

In [42]:
model = Models()
clf = model.logistic_regression_model()

In [56]:
def run_wrapper(params):
    s = time()
    auc = run_test( params )
    
    print
    print "Area Under Curve:", auc
    print "elapsed: {}s \n".format( int( round( time() - s )))
    
    return auc

def run_test( params ):
    C, class_weight, penalty = params
    
    clf.fit( X_train, y_train )
    p = clf.predict_proba( X_validation )[:, 1]
    
    auc = roc_auc_score( y_validation, p )
    
    return auc 

In [57]:
space = (
        hp.choice( 'C', [0.1, 0.5, 1.0, 3.0]),
        hp.choice( 'class_weight', [None, 'auto']),
        hp.choice( 'penalty', ['l2', 'l1'])
)

In [ ]:
start_time = time()
best = fmin( run_wrapper, space, 
             algo = tpe.suggest, max_evals=3)
end_time = time()

print 'Seconds passed:', int(round( end_time - start_time ))

print best